In [9]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.agents import initialize_agent, Tool

In [10]:
loader = PyPDFLoader("data/ICAR Annual Report 2023-24-english.pdf")
documents = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
docs = splitter.split_documents(documents)

In [11]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [14]:
vectorstore = Chroma.from_documents(
    docs,
    embeddings,
    persist_directory="./chroma_db_nomic"
)

retriever = vectorstore.as_retriever()

In [15]:
llm = ChatOllama(model="llama3.2", temperature=0)

In [16]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

In [17]:
query = "my rice crop isn't growing well due to bad climate what to do?"
print("Answer:", rag_chain.run(query))

/tmp/ipykernel_6294/223886074.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print("Answer:", rag_chain.run(query))


Answer: I don't know, but I can try to help you brainstorm some potential solutions.

Based on the text, it seems like there are some climate-smart interventions that have been identified for rice crops in your region. These include:

* Rice Puddling with N 120 kg and P 2O5
* Using a consortium of four bacteria to improve germination, vegetative growth, and yield under low moisture conditions

It's also mentioned that there are climate-smart options available for other crops like chickpea, wheat, soybean, and greengram.

You may want to consider consulting with local experts or farmers who have experience with these interventions. They can provide more specific advice based on your region's climate and soil conditions.

Additionally, you could try searching online for "climate-smart rice farming" or "rice crop management in [your region]" to see if there are any other resources available that might be helpful.

I hope this helps! Let me know if you have any other questions.


In [18]:
def get_weather(query: str):
    # Placeholder function
    return "Today's rainfall is 10mm, temperature is 28°C."

In [19]:
tools = [
    Tool(name="Knowledge Base", func=rag_chain.run, description="Look up agricultural advice"),
    Tool(name="Weather API", func=get_weather, description="Fetch latest weather data"),
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run("Given today's rainfall, should I irrigate my wheat field?")

/tmp/ipykernel_6294/622660081.py:6: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)




> Entering new AgentExecutor chain...
Question: Given today's rainfall, should I irrigate my wheat field?

Action: Knowledge Base
Action Input: "irrigation" and "wheat field"
Observation: I don't know the specific details about irrigation for a wheat field in the Tarai region. However, I can tell you that Nicra Pusa Wheat 3411 is a suitable variety for this region, with characteristics such as high iron content (40.6 ppm) and high sedimentation value (61.8 ml), making it resistant to yellow and brown rust.
Thought:Question: Given today's rainfall, should I irrigate my wheat field?

Action: Knowledge Base
Action Input: "irrigation" and "wheat field"

Observation: I don't know the answer to your question about "irrigation" and "wheat field". Can you provide more context or clarify what you are looking for? I'll do my best to help.
Thought:Question: Given today's rainfall, should I irrigate my wheat field?

Thought: Question: Given today's rainfall, should I irrigate my wheat field?

Ac

'Agent stopped due to iteration limit or time limit.'